In [173]:
import numpy as np
import random
import copy

In [174]:
# PLA 
class CyclePLA(object):
    def __init__(self,dimension,count,epsilon):
        self.count=count
        self.dimension=dimension
        self.epsilon=epsilon
        
    #get data
    def get_data_matrix(self,path):
        training_set = open(path)
        x_train=np.zeros((self.count,self.dimension + 1)) #初始化训练集
        y_train=np.zeros((self.count,1)) #初始化训练集标签
        x=[]
        x_count = 0
        for line in training_set:
            #在第一列先加入x0
            x.append(1)
            for s in line.split(' '):
                if len(s.split('\t')) == 1: #为了不考虑最后一个带转移符\n的元素
                    x.append(float(s))
                else:
                    x.append(float(s.split('\t')[0]))
                    y_train[x_count,0]=int(s.split('\t')[1].strip())
            x_train[x_count,:]=x
            x=[]
            x_count+=1
        return x_train,y_train
    
    def PLA(self,path):
        iteration=0   #计算一下循环的次数
        x_train,y_train=self.get_data_matrix(path)
        w=np.zeros(self.dimension+1)
        #进入循环直到所有的点都正确分类
        verify=True
        while verify:
            verify=False
            for i in range(self.count):
                if np.dot(w,x_train[i,:]) * y_train[i,0] <= 0:
                    w=w+self.epsilon*y_train[i,0]*x_train[i,:]
                    verify=True
                    iteration+=1
        return w,iteration

if __name__ == '__main__':
    perception = CyclePLA(4,400,1)
    print(perception.PLA("hw1_15_train.dat"))
                
            
        
        
        

(array([-3.       ,  3.0841436, -1.583081 ,  2.391305 ,  4.5287635]), 45)


In [176]:
# Random Cycle PLA
class RandomCyclePLA(object):
    def __init__(self,dimension,count,epsilon):
        self.count=count
        self.dimension=dimension
        self.epsilon=epsilon
        
    #get data
    def get_random_list(self,path):
        training_set = open(path)
        random_list=[]
        x=[]
        for line in training_set:
            #在第一列先加入x0
            x.append(1)
            for s in line.split(' '):
                if len(s.split('\t')) == 1: #为了不考虑最后一个带转移符\n的元素
                    x.append(float(s))
                else:
                    x.append(float(s.split('\t')[0]))
                    x.append(float(s.split('\t')[1]))
            random_list.append(x)
            random.shuffle(random_list)
            x=[]
        return random_list
    
    def get_data_matrix(self,path):
        random_list=self.get_random_list(path)
        x_train=np.zeros((self.count,self.dimension + 1)) #初始化训练集
        y_train=np.zeros((self.count,1)) #初始化训练集标签
        for i in range(len(random_list)):
            x_train[i,0]=1
            x_train[i,1:5]=random_list[i][1:5]
            y_train[i,0]=random_list[i][5]
        return x_train,y_train
    
    def PLA(self,path):
        a=[]
        for i in range(20):
            iteration=0   #计算一下循环的次数
            x_train,y_train=self.get_data_matrix(path)
            w=np.zeros(self.dimension+1)
            #进入循环直到所有的点都正确分类
            verify=True
            while verify:
                verify=False
                for i in range(self.count):
                    if np.dot(w,x_train[i,:]) * y_train[i,0] <= 0:
                        w=w+self.epsilon*y_train[i,0]*x_train[i,:]
                        verify=True
                        iteration+=1
            a.append(iteration)
        average=0
        for x in a:
            average+=x
        average=average/len(a)
        return average

if __name__ == '__main__':
    perception = RandomCyclePLA(4,400,0.5)
    print(perception.PLA("hw1_15_train.dat"))


41.05


In [206]:
# PLA-pocket 
class PocketCyclePLA(object):
    def __init__(self,dimension,train_count,test_count,epsilon):
        self.dimension=dimension
        self.test_count=test_count
        self.train_count=train_count
        self.epsilon=epsilon
        
    #get data
    def get_random_list(self,path):
        training_set = open(path)
        random_list=[]
        x=[]
        for line in training_set:
            #在第一列先加入x0
            x.append(1)
            for s in line.split(' '):
                if len(s.split('\t')) == 1: #为了不考虑最后一个带转移符\n的元素
                    x.append(float(s))
                else:
                    x.append(float(s.split('\t')[0]))
                    x.append(float(s.split('\t')[1]))
            random_list.append(x)
            random.shuffle(random_list)
            x=[]
        return random_list
    
    def get_train_data_matrix(self,train_path):
        random_list=self.get_random_list(train_path)
        x_train=np.zeros((self.train_count,self.dimension + 1)) #初始化训练集
        y_train=np.zeros((self.train_count,1)) #初始化训练集标签
        for i in range(len(random_list)):
            x_train[i,0]=1
            x_train[i,1:5]=random_list[i][1:5]
            y_train[i,0]=random_list[i][5]
        return x_train,y_train
    
    def get_test_data_matrix(self,test_path):
        test_set = open(test_path)
        x_test=np.zeros((self.test_count,self.dimension + 1)) #初始化训练集
        y_test=np.zeros((self.test_count,1)) #初始化训练集标签
        x=[]
        x_count = 0
        for line in test_set:
            #在第一列先加入x0
            x.append(1)
            for s in line.split(' '):
                if len(s.split('\t')) == 1: #为了不考虑最后一个带转移符\n的元素
                    x.append(float(s))
                else:
                    x.append(float(s.split('\t')[0]))
                    y_test[x_count,0]=int(s.split('\t')[1].strip())
            x_test[x_count,:]=x
            x=[]
            x_count+=1
        return x_test,y_test
    
    def PLA(self,train_path):
        best_w=np.zeros((self.dimension+1,1))    #保存最好的w
        best_count=self.train_count             #保存最少的错误数num
        count=0       #计算一下w修改的次数count
        x_train,y_train=self.get_train_data_matrix(train_path)
        w=np.zeros((self.dimension+1,1))
        #进入循环直到所有的点都正确分类
        for i in range(self.train_count):
            if np.dot(x_train[i,:], w)[0] * y_train[i, 0] <= 0:
                w += self.epsilon*y_train[i,0]*x_train[i,:].reshape(5,1)
                count+=1     #修改次数加一
                num=0
                for j in range(self.train_count):
                    if np.dot(x_train[j,:], w)[0] * y_train[j, 0] <= 0:
                        num+=1
                if num < best_count:
                    best_count=num
                    best_w=copy.deepcopy(w)
                if count == 50:
                    break
        return best_w
    
    def test_error(self,train_path,test_path):
        best_w=self.PLA(train_path)
        x_test,y_test=self.get_test_data_matrix(test_path)
        count=0
        for j in range(self.test_count):
            if np.dot(x_test[j,:],best_w)[0] * y_test[j, 0] <= 0:
                 count+=1
        return count/self.test_count

    
if __name__ == '__main__':
    avg_err_rate=0
    for i in range(20):
        perception = PocketCyclePLA(4,500,500,0.5)
        avg_err_rate+=perception.test_error("hw1_18_train.dat","hw1_18_test.dat")
    print(avg_err_rate/20.0)


0.126
